In [2]:
!git clone -b unet-gan https://github.com/hoangbros03/GAN-implement.git
!pip install torch-fidelity

# https://drive.google.com/file/d/1_0wdwSiL1dp-ewkVK2RD663fqmuL36BI/view?usp=sharing
# from zipfile import ZipFile
# with ZipFile("/kaggle/input/celeb-a/img_align_celeba", 'r') as zObject:
#     zObject.extractall(path="celeb-a")

Cloning into 'GAN-implement'...
remote: Enumerating objects: 20566, done.
remote: Counting objects: 100% (308/308), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 20566 (delta 148), reused 268 (delta 114), pack-reused 20258
Receiving objects: 100% (20566/20566), 547.37 MiB | 41.60 MiB/s, done.
Resolving deltas: 100% (149/149), done.


In [3]:
import os
os.chdir('GAN-implement/src/cnn_gan')

In [4]:
from train import Trainer
from reader import Reader
import torch
import torch.nn as nn

In [5]:
os.chdir('..')
os.chdir('..')
os.chdir('evaluate')
from evaluate import convert_dataloader_to_sample, frechet_inception_distance, convert_rgb, unnormalize

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
dataroot = "/kaggle/input/celeb-a-data"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
reader = Reader(dataroot=dataroot, batch_size=32, workers=2, image_size=64)
dataloader = reader.path_to_dataloader()

In [7]:
trainer = Trainer(device=device)
trainer.train_loop(dataloader, num_epochs=3, D_G_train_proportion=3)

100%|██████████| 3/3 [21:16<00:00, 425.47s/it]


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision.utils as vutils

# Plot the fake images from the last epoch
plt.figure(figsize=(15, 15))
plt.axis("off")
plt.title("Generated Images")
plt.imshow(np.transpose(trainer.img_list[800],(1,2,0)))
plt.savefig('gan-1')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(trainer.G_losses,label="G")
plt.plot(trainer.D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [8]:
gt = convert_dataloader_to_sample(dataloader)

In [9]:
gt = convert_dataloader_to_sample(dataloader)
for i in range(0, len(trainer.pred), 100):
    print(f"Iteration {i}")
    print(frechet_inception_distance(unnormalize(trainer.pred[i]), unnormalize(gt)))

Iteration 0


Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/weights-inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:00<00:00, 276MB/s]


tensor(299.8454)
Iteration 100
tensor(44.6134)
Iteration 200
tensor(16.2810)
Iteration 300
tensor(8.0392)
Iteration 400
tensor(5.3764)
Iteration 500
tensor(4.3220)
Iteration 600
tensor(5.0984)
Iteration 700
tensor(3.6818)
Iteration 800
tensor(2.2787)
Iteration 900
tensor(2.1378)
Iteration 1000
tensor(2.4979)
Iteration 1100
tensor(1.4230)
Iteration 1200
tensor(1.7613)
Iteration 1300
tensor(3.3765)
Iteration 1400
tensor(2.6248)
Iteration 1500
tensor(3.3654)
Iteration 1600
tensor(2.3786)
Iteration 1700
tensor(1.2274)
Iteration 1800
tensor(3.5547)
Iteration 1900
tensor(1.0956)
